In [1]:
%matplotlib inline
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import datetime

In [6]:
##Dict of session:uri
FMlinkList = {'August':'https://www.commonsense.org/education/privacy/blog/digital-redlining-access-privacy',
        'September':'http://dmlcentral.net/speculative-design-for-emergent-learning-taking-risks/',
        'October':'http://marginalsyllab.us/wp-content/uploads/2016/08/PWFlow-Intro.pdf',
        'November':'https://helenbeetham.com/2016/11/14/ed-tech-and-the-circus-of-unreason/',
        'January':'http://marginalsyllab.us/the-school-and-social-progress-by-john-dewey/',
        'February':'http://marginalsyllab.us/preface-to-research-writing-rewired-lessons-that-ground-students-digital-learning-by-dawn-reed-and-troy-hicks/',
        'FebruaryJCI':'http://cognitionandinstruction.com/engagements-the-learning-sciences-in-a-new-era-of-u-s-nationalism/',
        'March':'https://www.colorlines.com/articles/how-can-white-teachers-do-better-urban-kids-color',
        'April':'http://educatorinnovator.org/between-storytelling-and-surveillance-the-precarious-public-of-american-muslim-youth/'}

In [9]:
##Create empty dataFrame, and append data with the def scrapeAPI()
dfAllupdate = pd.DataFrame(columns=['created','document.title','group','id','links.html',
       'links.incontext','links.json','permissions.admin',
       'permissions.delete','permissions.read','permissions.update',
       'references','tags','target','text','updated','uri','user','session'])
for session,link in FMlinkList.items():
    df = scrapeAPI(session,link)
    dfAllupdate = dfAllupdate.append(df)
print('Total Records: ',len(dfAllupdate))

April
http://educatorinnovator.org/between-storytelling-and-surveillance-the-precarious-public-of-american-muslim-youth/
Number of Records Found for this Session:  71
Scraped:
0
20
40
60
71
March
https://www.colorlines.com/articles/how-can-white-teachers-do-better-urban-kids-color
Number of Records Found for this Session:  42
Scraped:
0
20
40
42
Total Records:  113


In [21]:
dateNow = datetime.datetime.now().strftime('%m%d%y_%H%M')
print(dateNow)
!mkdir $dateNow
#Build Excel sheet for onodo.org visualization
for session in FMlinkList:
    print('Month: ',session)
    print('Total: ',len(dfAllupdate[dfAllupdate['session']==session]))
    print('Annotations: ',len(dfAllupdate[(dfAllupdate['session']==session
                                                  )&(dfAllupdate['references'].isnull())]))
    print('Replies: ',len(dfAllupdate[(dfAllupdate['session']==session
                                                  )&(dfAllupdate['references'].notnull())]))
    print('')
    SNAnodes = dfAllupdate[dfAllupdate['session']==session].drop_duplicates(
        'user')['user'].str[5:].str[:-12].to_frame()
    SNAnodes.reset_index(inplace=True,drop=True)
    SNAnodes.rename(columns={'user':'Name'},inplace=True)
    
    reviewDataSet = dfAllupdate[dfAllupdate['session']==session
                           ][['created','user','id','group','tags',
                              'text','references','target']]
    if session is 'October':
        reviewDataSet['Anchor text'] = dfAllupdate[
            dfAllupdate['session']==session]['target'].map(lambda x: extractJSONOct(x[0]))
    else:
        reviewDataSet['Anchor text'] = dfAllupdate[
            dfAllupdate['session']==session]['target'].map(lambda x: extractJSON(x[0]))
    reviewDataSet['Parent'] = np.where(reviewDataSet['references'].isnull(),True,False)
    reviewDataSet['Child'] = np.where(reviewDataSet['references'].notnull(),True,False)
    reviewDataSet['created'] = reviewDataSet['created'].map(lambda x: x.strftime(
            '%m/%d/%Y %H:%M'))
    reviewDataSet['user'] = reviewDataSet['user'].str[5:].str[:-12]
    
    SNALinks = dfAllupdate[dfAllupdate['session']==session][['id','user','references']]
    SNALinks['Target'] = SNALinks['references'].map(
        lambda x: findUser(x[-1]),na_action='ignore')
    SNALinks['Target'].fillna('Text',inplace=True)
    SNALinks['Source'] = SNALinks['user'].str[5:].str[:-12]
    SNALinks['Type'] = np.where(SNALinks['Target']=='Text','annotation','reply')
    SNALinks['Directed'] = 1
    sessionDir = dateNow + '/' + session
    !mkdir $sessionDir
    writer = pd.ExcelWriter(sessionDir+'/MarginalSyllabus'+session+'.xlsx')
    SNAnodes.to_excel(writer,sheet_name='nodes',index=False)
    SNALinks[['Source','Type','Target','Directed']].to_excel(writer,sheet_name='Relations',index=False)
    reviewDataSet[['created','user','id','group',
                   'tags','Anchor text','text',
                   'Parent','Child']].to_excel(writer,sheet_name='DataReview',index=False)
    writer.save()

052917_1620
Month:  April
Total:  71
Annotations:  42
Replies:  29

Month:  March
Total:  42
Annotations:  32
Replies:  10



In [10]:
dateNow = datetime.datetime.now().strftime('%m%d%y_%H%M')
print(dateNow)
!mkdir $dateNow
#Build the Graphs and tables for the GDrive
for session in FMlinkList:
    Table1 = pd.DataFrame(index=list(dfAllupdate[dfAllupdate['session'] == session
                                            ].drop_duplicates('user')['user']))
    Table1['annotations'] = dfAllupdate[(dfAllupdate['session'] == session)&
                                (dfAllupdate['references'].isnull())
                               ].groupby('user').size()
    Table1['replies'] = dfAllupdate[(dfAllupdate['session'] == session)&
                                (~dfAllupdate['references'].isnull())
                               ].groupby('user').size()
    Table1.fillna(0,inplace=True)
    Table1.index = Table1.index.str[5:].str[:-12]
    Table1['total'] = Table1['replies'] + Table1['annotations']
    Table1.name = 'Table1'
    Graph1 = Table1.T
    Graph1.name = 'Graph1'
    
    Graph3 = dfAllupdate[(dfAllupdate['session'] == session)
                                ].groupby([pd.TimeGrouper('15T'),'user']).size().to_frame()
    Graph3.reset_index(level='user',inplace=True)
    Graph3 = pd.pivot_table(Graph3,columns='user',values=0,
                                         index=pd.Grouper(freq='15T'),fill_value=0)
    Graph3 = Graph3.rename(columns=lambda x: x[5:][:-12])
    Graph3.name = 'Graph3'

    graphEntireConversation = dfAllupdate[(dfAllupdate['session'] == session)
                                ].groupby([pd.TimeGrouper('D'),'user']).size().to_frame()
    graphEntireConversation.reset_index(level='user',inplace=True)
    graphEntireConversation['Total'] = graphEntireConversation.sum(axis=1)
    graphEntireConversation = pd.pivot_table(graphEntireConversation,columns='user',values=0,
                                         index=pd.Grouper(freq='D'),fill_value=0)
    graphEntireConversation = graphEntireConversation.rename(columns=lambda x: x[5:][:-12])
    graphEntireConversation['Total'] = graphEntireConversation.sum(axis=1)
    graphEntireConversation.name = 'graphEntireConversation'
    
    sessionDir = dateNow + '/' + session
    saveList = [Table1,Graph1,Graph3,graphEntireConversation]
    !mkdir $sessionDir
    print(sessionDir)
    for file in saveList:
        print(sessionDir + '/' + file.name + '.csv')
        print(sessionDir + '/' + file.name + '.pkl')
        file.to_csv(sessionDir + '/' + file.name + '.csv')
        file.to_pickle(sessionDir + '/' + file.name + '.pkl')

052917_1321
052917_1321/April
052917_1321/April/Table1.csv
052917_1321/April/Table1.pkl
052917_1321/April/Graph1.csv
052917_1321/April/Graph1.pkl
052917_1321/April/Graph3.csv
052917_1321/April/Graph3.pkl
052917_1321/April/graphEntireConversation.csv
052917_1321/April/graphEntireConversation.pkl
052917_1321/March
052917_1321/March/Table1.csv
052917_1321/March/Table1.pkl
052917_1321/March/Graph1.csv
052917_1321/March/Graph1.pkl
052917_1321/March/Graph3.csv
052917_1321/March/Graph3.pkl
052917_1321/March/graphEntireConversation.csv
052917_1321/March/graphEntireConversation.pkl


In [11]:
#Monthly Annotations, revisions and percentage of each
#Only prints values
for revMonth in FMlinkList:
    revisions = len(dfAllupdate[(dfAllupdate['session'] == revMonth)&
          (dfAllupdate['updated'] - dfAllupdate['created'] > pd.Timedelta(1,unit='s'))])
    totalAnnotations = len(dfAllupdate[dfAllupdate['session'] == revMonth])
    revPercent = (revisions / totalAnnotations) * 100
    print('month:\t\t',revMonth)
    print('revisions:\t',revisions)
    print('total:\t\t',totalAnnotations)
    print('percent:\t',revPercent)
    print('')

month:		 April
revisions:	 23
total:		 71
percent:	 32.3943661971831

month:		 March
revisions:	 3
total:		 42
percent:	 7.142857142857142



In [19]:
#Finds anchor text 'exact' in json, Oct JSON blob is different
def extractJSONOct(row):#for Oct
    if len(row) > 1:
        return row['selector'][1]['exact']
    else:
        return np.NaN
def extractJSON(row):#for months except Oct
    if len(row) > 1:
        return row['selector'][3]['exact']
    else:
        return np.NaN
def findUser(ref):
    #return dfAllupdate[dfAllupdate['id']==ref]['user'].str[5:].str[:-12][0]
    if len(SNALinks[SNALinks['id'] == ref]) is 0:
        return np.NaN
    else:
        return SNALinks[SNALinks['id']==ref]['user'].str[5:].str[:-12][0]

In [4]:
#Build dataframe from hypothes.is API
def scrapeAPI(session,uri):
    n = 0
    payload = {'url':uri}
    r = requests.get('https://hypothes.is/api/search',params=payload)
    df = pd.DataFrame(columns=['created', 'document.title', 'group', 'id', 'links.html',
       'links.incontext', 'links.json', 'permissions.admin',
       'permissions.delete', 'permissions.read', 'permissions.update',
       'references', 'tags', 'target', 'text', 'updated', 'uri', 'user'])
    total = r.json()['total']
    print(session)
    print(uri)
    print('Number of Records Found for this Session: ',total)
    print('Scraped:')
    while n < total:
        print(n)
        df = df.append(json_normalize(r.json()['rows']))
        n = len(df)
        payload = {
            'url':uri,
            'offset':n}
        r = requests.get('https://hypothes.is/api/search',params=payload)
    print(n)
    df['session'] = session
    df['updated'] = pd.to_datetime(df['updated'])
    df['created'] = pd.to_datetime(df['created'])
    df = df.set_index(df['created'])
    df.index = df.index.tz_localize('UTC').tz_convert('US/Mountain')
    return df